In [88]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow import keras
import numpy as np
from os import listdir
from os.path import isfile, join
import math

In [89]:
class SequenceGenerator(keras.utils.Sequence):
    """
    A keras Sequence to be used as an image generator for the model.
    """

    def __init__(self, x, y, batchsize):
        self.x, self.y, self.batchsize = x, y, batchsize

    def __len__(self):
        return math.ceil(len(self.x) / self.batchsize)

    def __getitem__(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])
        
        # open x image names, resize, normalise and make a numpy array
        batch_x = np.array([np.asarray(Image.open(file_name).resize((299, 299))) for file_name in x_names]) / 255.0

        return batch_x, y_names

    def num_classes(self):
        ret = []
        for cat in self.y:
            if cat not in ret:
                ret.append(cat)
        return len(ret)

In [90]:
def get_filenames_from_dir(directory, num_categories):
    x = []
    y = []
    category_count = 0
    for category in listdir(directory):
        for file in listdir("{0}/{1}".format(directory, category)):
            x.append("{0}/{1}/{2}".format(directory, category, file))
            y.append(category_count)
        category_count += 1
        if category_count == num_categories:
            break

    return x, y

In [91]:
# makes arrays of the images and label names
x_names, y_names = get_filenames_from_dir("database", 10)

# 15% of all the images are set aside as the test set
x_train_val, x_test, y_train_val, y_test = train_test_split(x_names, y_names, test_size=0.15, random_state=42)

# 17% of the non-test images are set aside as the validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.17, random_state=42)

In [92]:
# make generators with batch size 4 for each set
train_gen = SequenceGenerator(x_train, y_train, 1)
val_gen = SequenceGenerator(x_val, y_val, 1)
test_gen = SequenceGenerator(x_test, y_test, 1)

In [94]:
base_model = InceptionV3(include_top=False, weights="imagenet")
for layer in base_model.layers:
    layer.trainable = False

predictions = keras.layers.GlobalAveragePooling2D()(base_model.output)
#predictions = keras.layers.Dense(1024, activation='relu')(predictions)
predictions = keras.layers.Dense(train_gen.num_classes(), activation='softmax')(predictions)
model = keras.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, epochs=10)

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,80,192] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]